Earthquakes

In [1]:
#Run this first
import obspy
import numpy as np
from obspy.clients.fdsn.client import Client
import pandas as pd
import geopandas as gpd
from collections import defaultdict

import matplotlib.cm as cm
from matplotlib.pyplot import figure, show, rc
from matplotlib import pyplot as plt
import matplotlib
import plotly.express as px
import cartopy.crs as ccrs         # to plot maps with different projections
import cartopy.feature as cfeature # to plot coastlines, land, borders, etc.

from scipy.interpolate import griddata   #interpolate irregularly spaced data onto a grid

import alphashape
from descartes import PolygonPatch
import imageio

from scipy.interpolate import griddata   #interpolate irregularly spaced data onto a grid

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

In [2]:
#run this cell to define functions

def quakeML_Loader(filepath): #Loads quakeML file formats as Catalog
    # Make an obspy Catalog object from the QuakeML file.
    return obspy.core.event.read_events(filepath)
    #Copied from Maleen's code
    #Not sure this 1-line function actually needs to exist

def generate_summary(cat): #function that generates a summary table of data from a catalog of events for identification of best candidate events
    resource_ids = []
    magnitudes = []
    pick_counts =[]
    epi_lats = []
    epi_longs = []
    times = []
    depths = []
    for event in cat:
        resource_ids.append(event.resource_id)
        magnitudes.append(event.magnitudes[0].mag)
        pick_counts.append(len(event.picks))
        epi_lats.append(event.origins[0].latitude)
        epi_longs.append(event.origins[0].longitude)
        times.append(event.origins[0].time)
        depths.append(event.origins[0].depth)
    summary = pd.DataFrame(resource_ids, magnitudes).reset_index()
    summary.columns = ['magnitudes','resource_ids']
    summary['pick_counts'] = pick_counts #unsure why this needs to be done on its own but it works
    summary['epi_lats'] = epi_lats
    summary['epi_longs'] = epi_longs
    summary['times'] = times
    summary['depths'] = depths
    #these two lines can be edited for legibility and helpfulness
    print('Biggest earthquake was', summary.sort_values(by='magnitudes', ascending = False).get('resource_ids').iloc[0],'with magnitude',str(summary.sort_values(by='magnitudes', ascending = False).get('magnitudes').iloc[0])+'.')
    print('Best picked earthquake was', summary.sort_values(by='pick_counts', ascending = False).get('resource_ids').iloc[0],'with',summary.sort_values(by='pick_counts', ascending = False).get('pick_counts').iloc[0],'picks.')
    #TODO get index position(s) of most relevant quakes, but for our data set it's i = 170
    return summary

def quakeML_Reader(event):
    epicenter = (event.origins[0].latitude, event.origins[0].longitude) #stores event epicenter lat/lon as tuple
    magnitude = event.magnitudes[0].mag #stores event magnitude
    birthday = event.origins[0].time #stores time at which event occurs
    results = []
    for arrival in event.origins[0].arrivals: #goes through arrival data and notes phase, azimuthal angle, distance from epicenter and pick_id in 2D list
        phase = arrival.phase
        azi = arrival.azimuth
        dist = arrival.distance * 111 #there is a note in Maleen's code about this being in degrees
        pick_id = arrival.pick_id
        result = [phase, azi, dist, pick_id]
        results.append(result)
    arrivals = pd.DataFrame(results)
    arrivals.columns = ['phase','azimuth','distance','pick_id'] #makes DataFrame of arrivals data
    results = []
    for pick in event.picks: #goes through pick data and notes time of arrival, station data and pick_id in 2D list
        pick_id = pick.resource_id
        time = pick.time
        network_code = pick.waveform_id.network_code
        station_code = pick.waveform_id.station_code
        channel_code =pick.waveform_id.channel_code
        result = [pick_id, time, network_code, station_code, channel_code]
        results.append(result)
    picks = pd.DataFrame(results)
    picks.columns = ['pick_id','time','network_code','station_code','channel_code'] #makes DataFrame of picks data
    picks['travel_time'] = picks['time'] - birthday
    log = arrivals.merge(picks, left_on='pick_id', right_on='pick_id').sort_values(by='travel_time') #merges arrivals and picks data
    log['velocity'] = log['distance'] / log['travel_time']
    bonus = (epicenter, magnitude, birthday) #TODO integrate bonus

    #adds lat lon to picks
    #code borrowed from Zoe's explore_data.ipynb
    sta_list = np.unique(log.station_code)

    # Get all the info for those stations from IRIS
    network = ",".join((np.unique(log.network_code)).tolist())
    channel = ",".join((np.unique(log.channel_code)).tolist())
    station = ",".join((np.unique(log.station_code)).tolist())

    starttime = np.min(log['time'])
    endtime = np.max(log['time'])

    sta_metadata = Client("iris").get_stations(starttime=starttime,endtime=endtime,network=network,channel=channel,station=station,location='',level='response')

    station_locs = defaultdict(dict)
    for network in sta_metadata:
        for station in network:
            for chn in station:
                sid = f"{network.code}.{station.code}.{chn.location_code}.{chn.code[:-1]}" + chn.start_date.strftime('%Y%j')
                if sid in station_locs:
                    station_locs[sid]["component"] += f",{chn.code[-1]}"
                    station_locs[sid]["response"] += f",{chn.response.instrument_sensitivity.value:.2f}"
                else:
                    component = f"{chn.code[-1]}"
                    response = f"{chn.response.instrument_sensitivity.value:.2f}"
                    dtype = chn.response.instrument_sensitivity.input_units.lower()
                    tmp_dict = {}
                    tmp_dict["longitude"], tmp_dict["latitude"], tmp_dict["elevation(m)"] = (
                        chn.longitude,
                        chn.latitude,
                        chn.elevation,
                    )
                    tmp_dict["component"], tmp_dict["response"], tmp_dict["unit"] = component, response, dtype
                    tmp_dict["start_date"], tmp_dict["end_date"] = chn.start_date,chn.end_date
                    tmp_dict["network"], tmp_dict["station"] = network.code, station.code
                    station_locs[sid] = tmp_dict

    station_locs = pd.DataFrame.from_dict(station_locs,orient='index')
    station_locs["id"] = station_locs.index
    # Remove the date from ID
    station_locs['id']=station_locs['id'].str.slice(stop=-7)
    loc_log = log.merge(station_locs, left_on='station_code',right_on='station').drop(columns=['component','response','unit','start_date','end_date','id','network_code','station_code'])
    return loc_log

In [ ]:
cat = quakeML_Loader('XO_2019_01.quakeml') #loads quakeML file as events Catalog

#Should take about 1 minute to run on Noah's laptop, be patient

In [ ]:
summary = generate_summary(cat) #summarizes events and identifies most relevant
summary.iloc[170]

In [ ]:
event = cat[170] #biggest and most documented quake
log = quakeML_Reader(event) #be patient, this should take like 10s

In [ ]:
close_quakes = summary[(58 < summary['epi_lats']) & (59 > summary['epi_lats']) & (-156 < summary['epi_longs']) & (-155 > summary['epi_longs'])].index[0:].tolist()
len(close_quakes)

In [ ]:
close_log = quakeML_Reader(cat[57])
for i in close_quakes[1:]:
    event = cat[i]
    close_log = pd.concat([close_log,quakeML_Reader(event)])
close_log

We now have every pick with a latitude and longitude!  Analysis may ensue.

In [ ]:
#plot P-Waves azimuthal
log = close_log
print(log.shape[0])
plog = pd.DataFrame()
plog = plog.assign(azi=log[log.phase == 'P'].azimuth)
plog = plog.assign(velocity=log[log.phase == 'P'].velocity)
plog = plog[plog['velocity'] > 4]

def azi_binner(azi):
    if azi > 360-18/2:
        return 0
    else:
        return np.round(azi/18)*18
azis_binned = plog.get('azi').apply(azi_binner)
new_velocity = plog.get('velocity').apply(float)
plog = plog.assign(v = new_velocity)
plog = plog.assign(bins = azis_binned)
plog = plog.groupby('bins').mean().reset_index()

fig = px.line_polar(plog, r="v", theta="bins", line_close=True, range_r=[0,8], width =500, height =500)
fig.update_layout(title_text='P-Wave Direction (\N{DEGREE SIGN}) vs. Speed (km/s)', title_x=0.5)
fig.show()

In [ ]:
#plot P-Waves azimuthal
log = close_log
print(log.shape[0])
plog = pd.DataFrame()
plog = plog.assign(azi=log[log.phase == 'S'].azimuth)
plog = plog.assign(velocity=log[log.phase == 'S'].velocity)
plog = plog[plog['velocity'] > 2]

def azi_binner(azi):
    if azi > 360-18/2:
        return 0
    else:
        return np.round(azi/18)*18
azis_binned = plog.get('azi').apply(azi_binner)
new_velocity = plog.get('velocity').apply(float)
plog = plog.assign(v = new_velocity)
plog = plog.assign(bins = azis_binned)
plog = plog.groupby('bins').mean().reset_index()

fig = px.line_polar(plog, r="v", theta="bins", line_close=True, range_r=[0,8], width =500, height =500)
fig.update_layout(title_text='S-Wave Direction (\N{DEGREE SIGN}) vs. Speed (km/s)', title_x=0.5)
fig.show()

In [ ]:
#plot Waves azimuthal
log = close_log
print(log.shape[0])
slog = pd.DataFrame()
slog = slog.assign(azi=log[log.phase == 'S'].azimuth)
slog = slog.assign(velocity=log[log.phase == 'S'].velocity)
slog = slog[slog['velocity'] > 1]

def azi_binner(azi):
    if azi > 360-18/2:
        return 0
    else:
        return np.round(azi/18)*18
azis_binned = slog.get('azi').apply(azi_binner)
new_velocity = slog.get('velocity').apply(float)
slog = slog.assign(v = new_velocity)
slog = slog.assign(bins = azis_binned)
slog = slog.groupby('bins').mean().reset_index()

log = close_log
plog = pd.DataFrame()
plog = plog.assign(azi=log[log.phase == 'P'].azimuth)
plog = plog.assign(velocity=log[log.phase == 'P'].velocity)
plog = plog[plog['velocity'] > 4]

def azi_binner(azi):
    if azi > 360-18/2:
        return 0
    else:
        return np.round(azi/18)*18
azis_binned = plog.get('azi').apply(azi_binner)
new_velocity = plog.get('velocity').apply(float)
plog = plog.assign(v = new_velocity)
plog = plog.assign(bins = azis_binned)
plog = plog.groupby('bins').mean().reset_index()

flog = pd.concat([plog,slog], keys=['P','S']).reset_index()
flog['Wave'] = flog.level_0
fig = px.line_polar(flog, r="v", theta="bins", color ="Wave", line_close=True, range_r=[0,8], width =500, height =500)
fig.update_layout(title_text='Wave Direction (\N{DEGREE SIGN}) vs. Speed (km/s)', title_x=0.5)
fig.show()

#I'm not installing another package, mouseover the figure for a png download

In [ ]:
log = close_log
plog = pd.DataFrame()
plog = plog.assign(azi=log[log.phase == 'P'].azimuth)
plog = plog.assign(velocity=log[log.phase == 'P'].velocity)
plt.scatter(plog.azi,plog.velocity)

In [ ]:
#makes wave position GIFs
#should take a while to run
file_names = []

Lon_Honolulu=360-155.294; Lat_Honolulu=58.320

for t in range(32, 140):
    plog = log[log['phase'] == 'P']
    plog = plog[plog['travel_time'] < t]
    slog = log[log['phase'] == 'S']
    slog = slog[slog['travel_time'] < t]
    first_pick_time = int(log[log['phase'] == 'P'].travel_time.min())
    min_lat = log[log['phase'] == 'P'].latitude.min()
    min_lon = log[log['phase'] == 'P'].longitude.min()
    max_lat = log[log['phase'] == 'P'].latitude.max()
    max_lon = log[log['phase'] == 'P'].longitude.max()

    # data coordinates and values
    xp = plog.longitude
    yp = plog.latitude
    zp = plog.travel_time
    xs = slog.longitude
    ys = slog.latitude
    zs = slog.travel_time    
    

    # specify the desired grid to interpolate data
    numcols, numrows = 50, 50
    xip = np.linspace(xp.min() - 2, xp.max() + 2, numcols)
    yip = np.linspace(yp.min() - 2, yp.max() + 2, numrows)
    xip, yip = np.meshgrid(xip, yip)
    pzi = griddata((xp,yp),zp,(xip,yip), method='linear')  # interpolate data onto the specified grid
    
    xis = np.linspace(xs.min() - 2, xs.max() + 2, numcols)
    yis = np.linspace(ys.min() - 2, ys.max() + 2, numrows)
    xis, yis = np.meshgrid(xis, yis)
    szi = griddata((xs,ys),zs,(xis,yis), method='linear')  # interpolate data onto the specified grid

    plt.figure(figsize=(15,5))
    ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180)); #this specifies which projection to use
    ax.set_extent(( 360-180,360-135, 50,70), crs=ccrs.PlateCarree())
    ax.plot(Lon_Honolulu,Lat_Honolulu, 'r*', markersize=10, alpha=1,transform=ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.LAND)
    plt.title(str(plog.time.iloc[t-1]))
    gl = ax.gridlines(draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False
    
    levels = 100
    plt.contour(xip, yip, pzi, levels, colors = 'b', alpha = 0.1,
                 transform=ccrs.PlateCarree())
    plt.contour(xis, yis, szi, levels, colors = 'r', alpha = 0.1,
                 transform=ccrs.PlateCarree())
    
    file_name = 'stills/'+str(t-1)
    file_names.append(file_name)
    plt.savefig(file_name)
    plt.close()

images = []
for filename in file_names:
    images.append(imageio.imread(str(filename+'.png')))
imageio.mimsave('test.gif', images)

In [ ]:
#plots p waves
Lon_Honolulu=360-155.294; Lat_Honolulu=58.320

plog = log[log['phase'] == 'P']
x = plog.longitude
y = plog.latitude
z = plog.travel_time

# specify the desired grid to interpolate data
numcols, numrows = 100, 100
xi = np.linspace(x.min() - 2, x.max() + 2, numcols)
yi = np.linspace(y.min() - 1, y.max() + 1, numrows)
xi, yi = np.meshgrid(xi, yi)
zi = griddata((x,y),z,(xi,yi), method='linear')

plt.figure(figsize=(15,5))
levels = range(0, 160, 20)
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180)); #this specifies which projection to use
ax.set_extent(( 360-180,360-135, 50,70), crs=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.LAND)
cs = plt.contour(xi, yi, zi, levels, colors = 'k',
             transform=ccrs.PlateCarree())
plt.clabel(cs, inline=True, fontsize=10)
sc = ax.scatter(xi,yi, c = zi, cmap = 'hot', transform=ccrs.PlateCarree())
ax.scatter(xi,yi, cmap = 'tab20_r',alpha = 0.4)
#ax.scatter(x,y,5,transform=ccrs.PlateCarree())
plt.title('P Wave Travel Time')
ax.plot(Lon_Honolulu,Lat_Honolulu, 'r*', markersize=10, alpha=1,transform=ccrs.PlateCarree())
plt.colorbar(sc)



#plt.text(Lon_Honolulu-1,Lat_Honolulu-0.25,      # add a text label
         #'Honolulu',transform=ccrs.PlateCarree())

gl = ax.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False

file_name = 'travel_time_maps/p_wave'
#file_names.append(file_name)
plt.savefig(file_name)

In [ ]:
#plots S waves
Lon_Honolulu=360-155.294; Lat_Honolulu=58.320

plog = log[log['phase'] == 'S']
x = plog.longitude
y = plog.latitude
z = plog.travel_time

# specify the desired grid to interpolate data
numcols, numrows = 100, 100
xi = np.linspace(x.min() - 1, x.max() + 1, numcols)
yi = np.linspace(y.min() - 1, y.max() + 1, numrows)
xi, yi = np.meshgrid(xi, yi)
zi = griddata((x,y),z,(xi,yi), method='linear')

levels = range(0, 160, 20)
plt.figure(figsize=(15,5))
cs = ax.contour(xi,yi,zi)
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180)); #this specifies which projection to use
ax.set_extent(( 360-180,360-135, 50,70), crs=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.LAND)
cs = plt.contour(xi, yi, zi, levels, colors = 'k',
             transform=ccrs.PlateCarree())
plt.clabel(cs, inline=True, fontsize=10)
sc = ax.scatter(xi,yi, c = zi, cmap = 'hot', transform=ccrs.PlateCarree())
ax.scatter(xi,yi, c=zi, alpha = 0.4)
#ax.scatter(x,y,5,transform=ccrs.PlateCarree())
plt.title('S Wave Travel Time')
ax.plot(Lon_Honolulu,Lat_Honolulu, 'r*', markersize=10, alpha=1,transform=ccrs.PlateCarree())
plt.colorbar(sc)



#plt.text(Lon_Honolulu-1,Lat_Honolulu-0.25,      # add a text label
         #'Honolulu',transform=ccrs.PlateCarree())

gl = ax.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False

file_name = 'travel_time_maps/s_wave'
#file_names.append(file_name)
plt.savefig(file_name)

In [ ]:
#replicating Maleen's plot, some code is copied and modified
p_log = log[log['phase'] == 'P']
s_log = log[log['phase'] == 'S']

fig,ax = plt.subplots(1,1,figsize=[10,10])
ax.plot(p_log['travel_time'],p_log['distance'],'.')
ax.plot(s_log['travel_time'],s_log['distance'],'.')
ax.set_xlabel('Travel time (s)')
ax.set_ylabel('Distance (km)')
ax.legend(['P-waves','S-waves'])

# P-wave fit
m,b = np.polyfit(p_log['travel_time'].tolist(),p_log['distance'].tolist(),1)
x_p = np.linspace(0,150)
ax.plot(x_p,x_p*m + b,'-')
print('P-wave slope is '+str(m)+' km/s.')

# S-wave fit
m,b = np.polyfit(s_log['travel_time'].tolist(),s_log['distance'].tolist(),1)
x_p = np.linspace(0,150)
ax.plot(x_p,x_p*m + b,'-')
print('S-wave slope is '+str(m)+' km/s.')